In [103]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import country_converter as coco





In [80]:
url = 'https://www.thespike.gg/stats/players?rounds=300&region=all&date=all&map=all&agent=all&team_rank=30&series=all'
page = requests.get(url)
soup= BeautifulSoup(page.text, 'html.parser')

In [81]:
#parsed a list of all players
players = []
stats = soup.find_all('div', {'class': 'player-info'})
for i in stats:
    players.append(i.text)


In [82]:
statistics = []
idk = soup.find_all(class_ = 'number')
for stat in idk:
    statistics.append(stat.text)
print(statistics[10])

35%


In [88]:
#Trying to parse countries for every player in order we want to create geospatial data in tableau
countries = []
for i in range(733):
    country = stats[i].select('#stats-players table tr td.player a span.flag-icon')
    countries.append(str(country[0])[-11:-9]) 

In [90]:
#now we need to create a dictionary with player nickname as a key and stats as value
players_dict = {}
for i in range(len(players)):
    player = players[i]
    players_dict[player] = []
    players_dict[player].append(countries[i])
    for j in range(12):
        players_dict[player].append(statistics[i*12+j])
    
        


In [91]:
columns = ['Country', 'Rounds', 'Rating', 'ACS', "K/D", "ADR", 'KPR', 'DPR', 'APR', 'FBPR', 'FDPR', 'HS%', 'FBSR']
df = pd.DataFrame.from_dict(players_dict, orient='index', columns=columns)
df.index.name ='Player'

In [92]:
df.reset_index()

,Player,Country,Rounds,Rating,ACS,K/D,ADR,KPR,DPR,APR,FBPR,FDPR,HS%,FBSR
0,Ivy,co,522,1.57,306.4,1.95,192.8,1.10,0.56,0.24,0.22,0.08,35%,73%
1,mwzera,br,3089,1.36,286.8,1.40,175.2,0.99,0.71,0.24,0.18,0.10,17%,65%
2,cNed,tr,5644,1.36,270.3,1.44,162.7,0.96,0.67,0.18,0.19,0.11,24%,65%
3,something,ru,975,1.35,283.2,1.44,175.9,1.00,0.69,0.18,0.19,0.14,22%,58%
4,Jianna,am,410,1.35,276.7,1.38,182.7,0.91,0.66,0.35,0.15,0.07,18%,70%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,GamKa,kr,549,0.77,181.1,0.73,118.4,0.61,0.83,0.19,0.07,0.15,19%,33%
728,Momonga,bo,799,0.77,165.5,0.76,105.9,0.56,0.74,0.18,0.09,0.14,21%,38%
729,fRoD,us,338,0.76,201.6,1.03,129.5,0.69,0.67,0.25,0.10,0.09,23%,53%
730,depppx,br,676,0.73,147.8,0.77,98.4,0.51,0.67,0.27,0.04,0.07,14%,35%


In [109]:
#Converting iso-code country-names to standart
df['Country'] = coco.convert(names=df['Country'], to='name_short', not_found=None)

not found not found in regex
not found not found in regex
not found not found in regex
not found not found in regex
not found not found in regex
not found not found in regex
not found not found in regex
not found not found in regex
not found not found in regex
not found not found in regex


In [108]:
df.head()

,Country,Rounds,Rating,ACS,K/D,ADR,KPR,DPR,APR,FBPR,FDPR,HS%,FBSR
Player,,,,,,,,,,,,,
Ivy,Colombia,522,1.57,306.4,1.95,192.8,1.10,0.56,0.24,0.22,0.08,35%,73%
mwzera,Brazil,3089,1.36,286.8,1.40,175.2,0.99,0.71,0.24,0.18,0.10,17%,65%
cNed,Turkey,5644,1.36,270.3,1.44,162.7,0.96,0.67,0.18,0.19,0.11,24%,65%
something,Russia,975,1.35,283.2,1.44,175.9,1.00,0.69,0.18,0.19,0.14,22%,58%
Jianna,Armenia,410,1.35,276.7,1.38,182.7,0.91,0.66,0.35,0.15,0.07,18%,70%


In [110]:
df.to_csv('valorant_stats.csv', sep=',')